In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path_repo = expanduser("~/Documents/G3_2/regime-identification"); sys.path.append(path_repo)
path_file = expanduser("~/data/G3_2/regime-identification/simulation")
path = {}
for folder in ["data", "estimation", "score", "figure", "latex"]:
    path[folder] = f"{path_file}/{folder}"

In [15]:
import pandas as pd
import numpy as np

from numpy.random import RandomState
random_state = RandomState(0)

In [3]:
from regime.crsp import *

In [13]:
from regime.jump import *

In [4]:
import wrds

In [5]:
db = wrds.Connection()

Enter your WRDS username [yizhan]: olivershu
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
Loading library list...
Done


In [6]:
ret_ser = get_total_mkt_return_monthly_CRSP(db=db)

In [87]:
X = ret_ser[pd.Period("2000-01", "M"):pd.Period("2022-12", "M")].to_numpy()[:, np.newaxis]

In [12]:
from hmmlearn.hmm import GaussianHMM

In [74]:
model=GaussianHMM_model(2, random_state=random_state, min_covar=1e-6, covars_prior=1e-6, n_iter=300, tol=1e-6, )

In [75]:
model.fit(X)

GaussianHMM_model(random_state=RandomState(MT19937) at 0x14B78C440)

In [76]:
model.transmat_

array([[0.94579249, 0.05420751],
       [0.06060559, 0.93939441]])

In [77]:
model_3=GaussianHMM_model(3, n_init=30, random_state=random_state, min_covar=1e-6, covars_prior=1e-6, n_iter=300, tol=1e-6, )

In [88]:
model_3.fit(X)

GaussianHMM_model(n_components=3, n_init=30,
                  random_state=RandomState(MT19937) at 0x14B78C440)

In [98]:
print("trans matrix:")
pd.DataFrame(model_3.transmat_, index=index, columns=index).round(2)

trans matrix:


,State 0,State 1,State 2
State 0,0.23,0.29,0.48
State 1,0.00,0.94,0.06
State 2,0.30,0.00,0.70


In [99]:
print("means:")
pd.Series(model_3.means_, index=index)

means:


State 0    0.068238
State 1    0.014001
State 2   -0.032986
dtype: float64

In [100]:
print("vol:")
pd.Series(np.sqrt(model_3.covars_), index=index)

vol:


State 0    0.027229
State 1    0.023726
State 2    0.048495
dtype: float64

In [93]:
index=[f"State {i}" for i in range(3)]

In [23]:
perm = np.array([1, 0, 2])

In [27]:
model_3.transmat_[perm][:, perm]

array([[3.38746398e-01, 4.88224815e-01, 1.73028787e-01],
       [3.82531541e-01, 6.17468459e-01, 1.16030400e-25],
       [2.91455913e-10, 4.41052379e-01, 5.58947620e-01]])

In [ ]:
GaussianHMM(2, min_covar=1e-6, covars_prior=0.01, n_iter=300, tol=1e-6, )